In [ ]:
from pathlib import Path
from utils.pred_and_anno import fetch_predicted_bbox_by_image_id
from utils.meta_io import fetch_scene_object_by_image_id
from utils.intrinsic_fetcher import IntrinsicFetcher
import torch


# scene_by_image_id = fetch_scene_object_by_image_id('v2_3d')
# intrinsic_fetcher = IntrinsicFetcher()
# pred_bbox_by_image_id = fetch_predicted_bbox_by_image_id()
# seg_out_dir = Path('/home/junha/Downloads/segformer')
# exception_label_indices = [0, 3, 5]

In [2]:
scene_by_image_id_2d = fetch_scene_object_by_image_id('v2_2d')

In [7]:
image_id = '000001'
scene_by_image_id = torch.load('/home/junha/data/sunrefer/meta.pt')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
from compute_iou import PredictionVisualizer
vis = PredictionVisualizer(
        aabb=True,
        highlight=True,
        apply_seg_out_mask=False,
        verbose=True,
        bbox_2d_ratio=1.0,
        pcd_th_ratio=0.1,
        mask_ratio=0.8)
vis.compute_3d_bbox('000001_1_0')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [34]:
import numpy as np


def convert_orientedbbox2AABB(verts):
    x_min = np.min(verts[:, 0], axis=0)
    x_max = np.max(verts[:, 0], axis=0)
    y_min = np.min(verts[:, 1], axis=0)
    y_max = np.max(verts[:, 1], axis=0)
    z_min = np.min(verts[:, 2], axis=0)
    z_max = np.max(verts[:, 2], axis=0)
    cx = (x_min + x_max) / 2
    cy = (y_min + y_max) / 2
    cz = (z_min + z_max) / 2
    sx = x_max - x_min
    sy = y_max - y_min
    sz = z_max - z_min
    return cx, cy, cz, sx, sy, sz


def aabb_from_coeffs(bbox_3d):
    centroid = np.array(bbox_3d.centroid).reshape(-1, )
    basis = np.array(bbox_3d.basis)
    coeffs = np.array(bbox_3d.coeffs).reshape(-1, )
    vx = basis[0] * coeffs[0]
    vy = basis[1] * coeffs[1]
    vz = basis[2] * coeffs[2]

    cl = [[-1, -1, -1],
          [1, -1, -1],
          [-1, 1, -1],
          [1, 1, -1],
          [-1, -1, 1],
          [1, -1, 1],
          [-1, 1, 1],
          [1, 1, 1]]
    pl = [dx * vx + dy * vy + dz * vz for dx, dy, dz in cl]
    cx, cy, cz, sx, sy, sz = convert_orientedbbox2AABB(np.array(pl))
    cx += centroid[0]
    cy += centroid[1]
    cz += centroid[2]
    return cx, cy, cz, sx, sy, sz
    # cx, cy, cz, sx, sy, sz = convert_orientedbbox2AABB(np.array(pl))

In [63]:
from scripts.compute_iou import convert_orientedbbox2AABB

image_id = '000001'
pred_bbox_list = pred_bbox_by_image_id[image_id]
scene_3d = scene_by_image_id[image_id]
scene_2d = scene_by_image_id_2d[image_id]
for uniq_id, pred_bbox_2d in pred_bbox_list:
    image_id, object_id, anno_id = uniq_id.split('_')
    object_id = int(object_id)
    gt_bbox_3d = scene_3d.gt_3d_bbox[object_id]
    print(gt_bbox_3d.centroid)
    print(gt_bbox_3d.coeffs)
    print(gt_bbox_3d.basis)

    gt_bbox_2d = scene_2d.gt_2d_bbox[object_id]
    fx, fy, tx, ty = scene_3d.K[0, 0], scene_3d.K[1, 1], scene_3d.K[0, 2], scene_3d.K[1, 2]
    # print(uniq_id, gt_bbox_2d, gt_bbox_3d)
    # print(fx, fy, tx, ty)
    cx, cy, cz, sx, sy, sz = aabb_from_coeffs(gt_bbox_3d)
    cy, cz = -cz, cy
    sz, sy = sy, sz
    print(fx, fy, tx, ty)
    print(gt_bbox_2d.gt_bbox_2d)

    # px = (gt_bbox_3d.centroid @ np.transpose(scene_3d.extrinsics))[0]
    px = gt_bbox_3d.centroid[0]
    p0 = cx, cy, cz
    p1 = cx - 0.5 * sx, cy - 0.5 * cy, cz
    p2 = cx + 0.5 * sx, cy - 0.5 * cy, cz
    p3 = cx + 0.5 * sx, cy + 0.5 * cy, cz
    p4 = cx - 0.5 * sx, cy + 0.5 * cy, cz

    def projection(p):
        return p[0] / p[2] * fx + tx, p[1] / p[2] * fy + ty

    # qx = (px[0] / px[2]) * fx + tx, (px[1] / px[2]) * fy + ty
    # q0 = (p0[0] / p0[2]) * fx + tx, (p0[1] / p0[2]) * fy + ty
    # q1 = (p1[0] / p1[2]) * fx + tx, (p1[1] / p1[2]) * fy + ty
    # q2 = (p2[0] / p2[2]) * fx + tx, (p2[1] / p2[2]) * fy + ty
    # q3 = (p3[0] / p3[2]) * fx + tx, (p3[1] / p3[2]) * fy + ty
    # q4 = (p4[0] / p4[2]) * fx + tx, (p4[1] / p4[2]) * fy + ty
    qx = projection(px)
    q0 = projection(p0)
    q1 = projection(p1)
    q2 = projection(p2)
    q3 = projection(p3)
    q4 = projection(p4)
    print('qx', qx)
    print(q0)
    print(q1)
    print(q2)
    print(q3)
    print(q4)

[[-1.08636364  1.90454545 -0.14772727]]
[[0.78156708 0.35640538 1.05227273]]
[[ 0.10468478  0.99450545  0.        ]
 [-0.99478144  0.10202887  0.        ]
 [ 0.          0.          1.        ]]
529.5 529.5 365.0 265.0
[1, 1, 221, 529]
qx (4258.861537634391, -6561.476921863775)
(62.97016707492287, 306.07100239393367)
(-58.34725534464803, 285.53550119696683)
(184.28758949449374, 285.53550119696683)
(184.28758949449374, 326.6065035909005)
(-58.34725534464803, 326.6065035909005)
[[-1.08636364  1.90454545 -0.14772727]]
[[0.78156708 0.35640538 1.05227273]]
[[ 0.10468478  0.99450545  0.        ]
 [-0.99478144  0.10202887  0.        ]
 [ 0.          0.          1.        ]]
529.5 529.5 365.0 265.0
[1, 1, 221, 529]
qx (4258.861537634391, -6561.476921863775)
(62.97016707492287, 306.07100239393367)
(-58.34725534464803, 285.53550119696683)
(184.28758949449374, 285.53550119696683)
(184.28758949449374, 326.6065035909005)
(-58.34725534464803, 326.6065035909005)
[[-1.08636364  1.90454545 -0.14772727]